In [1]:
import sys
from pathlib import Path
src_path = Path.cwd().parent / 'src'
sys.path.append(str(src_path))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [2]:
from data_loader import load_data

# Set up data paths
data_path = Path.cwd().parent / "data" / "ag_news"
train_path = data_path / "train.csv"
test_path = data_path / "test.csv"

# Load training data
df_train = load_data(train_path)
X_train = (df_train["Title"] + " " + df_train["Description"]).values
y_train = df_train["Class Index"].values

# Load test data
df_test = load_data(test_path)
X_test = (df_test["Title"] + " " + df_test["Description"]).values
y_test = df_test["Class Index"].values

# Show training data
# df_train.head()

# Show test data
# df_test.head()


In [3]:
from vectorizer import vectorize_data_fit
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize data
vectorizer = TfidfVectorizer()
X_train_vect = vectorize_data_fit(vectorizer, X_train)

In [4]:
from model_trainer import train_model
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

# Train Logistic Regression Model
lg_model = LogisticRegression()
#lg_trained_model = train_model(lg_model, X_train_vect, y_train)

# Train Naive Bayes Model
nb_model = MultinomialNB()
nb_trained_model = train_model(nb_model, X_train_vect, y_train)

# Train Random Forest Model
rf_model = RandomForestClassifier()
#rf_trained_model = train_model(rf_model, X_train_vect, y_train)

In [5]:
# from perturbation import apply_perturbation

# # Save perturbed data to file

# X_data = X_test[:1000]
# level = 0.4

# perturbed_data = apply_perturbation(
#     X_data,
#     level,
#     save_path=f"perturbed_data/wordnet/perturbed_data_{level:.2f}.pkl"
# )



In [ ]:
from evaluator import evaluate_robustness

# Evaluation pipeline

# Number of samples to evaluate
X_sample = X_test[:1000]
y_sample = y_test[:1000]

# Define perturbation levels to test
perturbation_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# Add required metrics to evaluate
metrics = ["base_accuracy", "robustness_score", "effective_robustness", "mce", "pdr", "our_metric", "simple_mds", "max_mds", "new_mds"]

# Use filepath when using pertrubed data from files for faster testing
file_path = Path("perturbed_data/wordnet")

# Add model to evaluate (also used for plotting)
model = nb_trained_model

results, metrics_summary = evaluate_robustness(
    model,
    vectorizer,
    X_sample, 
    y_sample,
    perturbation_levels,
    metrics,
    file_path
)

# Do we need metric summary? - info given in table for each level instead
# print("*** Metrics Summary ***")
# for metric, value in metrics_summary.items():
#     print(f"{metric}: {value: .4f}")

results_df = pd.DataFrame(results)
print("*** Results Per Perturbation Level")
display(results_df)

# Save results for each model
# LR
#results_df.to_csv(Path("results/wordnet/lr.csv"), index=False)
# NB
results_df.to_csv(Path("results/wordnet/nb.csv"), index=False)
# RF
#results_df.to_csv(Path("results/wordnet/rf.csv"), index=False)


c:\Users\t440p\miniconda3\envs\thesis-py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


*** Results Per Perturbation Level


,perturbation level,accuracy,RS,mCE,PDR,MDS,SimpleMDS,MaxMDS,NewMDS
0,0.0,0.882,None,None,None,None,None,None,NaN
1,0.1,0.885,None,None,None,None,None,None,NaN
2,0.2,0.874,None,None,None,None,None,None,0.22
3,0.3,0.866,None,None,None,None,None,None,0.16
4,0.4,0.860,None,None,None,None,None,None,0.12
5,0.5,0.854,None,None,None,None,None,None,0.12
6,0.6,0.852,None,None,None,None,None,None,0.04
7,0.7,0.837,None,None,None,None,None,None,0.30
8,0.8,0.827,None,None,None,None,None,None,0.20
9,0.9,0.831,None,None,None,None,None,None,0.00


PLOTTING RESULTS

In [ ]:
# load csv files

#CHARSWAP
df_chars_lr = pd.read_csv(Path("results/charswap/lr.csv"))
df_chars_nb = pd.read_csv(Path("results/charswap/nb.csv"))
df_chars_rf = pd.read_csv(Path("results/charswap/rf.csv"))

#DELETION
df_del_lr = pd.read_csv(Path("results/deletion/lr.csv"))
df_del_nb = pd.read_csv(Path("results/deletion/nb.csv"))
df_del_rf = pd.read_csv(Path("results/deletion/rf.csv"))

#WORDNET
df_word_lr = pd.read_csv(Path("results/wordnet/lr.csv"))
df_word_nb = pd.read_csv(Path("results/wordnet/nb.csv"))
df_word_rf = pd.read_csv(Path("results/wordnet/rf.csv"))


# SUBSETS TO PLOT

# LR all accuracy/perturbation
plt.plot(df_chars_lr["perturbation level"], df_chars_lr["accuracy"], marker="o", linestyle= "-", color="red", label="Charswap")
plt.plot(df_word_lr["perturbation level"], df_word_lr["accuracy"], marker="v", linestyle= "-", color="blue", label="WordNet")
plt.plot(df_del_lr["perturbation level"], df_del_lr["accuracy"], marker="s", linestyle= "-", color="green", label="Deletion")
plt.xlabel("Perturbation level")
plt.ylabel("Accuracy")
plt.title("Logistic Regression All Perturbation Types")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(0.4, 1)
plt.grid()
plt.show()


# NB all accuracy/perturbation
plt.plot(df_chars_nb["perturbation level"], df_chars_nb["accuracy"], marker="o", linestyle= "-", color="red", label="Charswap")
plt.plot(df_word_nb["perturbation level"], df_word_nb["accuracy"], marker="v", linestyle= "-", color="blue", label="WordNet")
plt.plot(df_del_nb["perturbation level"], df_del_nb["accuracy"], marker="s", linestyle= "-", color="green", label="Deletion")
plt.xlabel("Perturbation level")
plt.ylabel("Accuracy")
plt.title("Naive Bayes All Perturbation Types")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(0.4, 1)
plt.grid()
plt.show()

# RF all
plt.plot(df_chars_rf["perturbation level"], df_chars_rf["accuracy"], marker="o", linestyle= "-", color="red", label="Charswap")
plt.plot(df_word_rf["perturbation level"], df_word_rf["accuracy"], marker="v", linestyle= "-", color="blue", label="WordNet")
plt.plot(df_del_rf["perturbation level"], df_del_rf["accuracy"], marker="s", linestyle= "-", color="green", label="Deletion")
plt.xlabel("Perturbation level")
plt.ylabel("Accuracy")
plt.title("Random Forest All Perturbation Types")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(0.4, 1)
plt.grid()
plt.show()


# CHARSWAP
plt.plot(df_chars_lr["perturbation level"], df_chars_lr["accuracy"], marker="o", linestyle= "-", color="orange", label="LR")
plt.plot(df_chars_nb["perturbation level"], df_chars_nb["accuracy"], marker="v", linestyle= "-", color="gray", label="NB")
plt.plot(df_chars_rf["perturbation level"], df_chars_rf["accuracy"], marker="s", linestyle= "-", color="brown", label="RF")
plt.xlabel("Perturbation level")
plt.ylabel("Accuracy")
plt.title("Charswap")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(0.4, 1)
plt.grid()
plt.show()

# WORDNET
plt.plot(df_word_lr["perturbation level"], df_word_lr["accuracy"], marker="o", linestyle= "-", color="orange", label="LR")
plt.plot(df_word_nb["perturbation level"], df_word_nb["accuracy"], marker="v", linestyle= "-", color="gray", label="NB")
plt.plot(df_word_rf["perturbation level"], df_word_rf["accuracy"], marker="s", linestyle= "-", color="brown", label="RF")
plt.xlabel("Perturbation level")
plt.ylabel("Accuracy")
plt.title("WordNet")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(0.4, 1)
plt.grid()
plt.show()

# DELETION
plt.plot(df_del_lr["perturbation level"], df_del_lr["accuracy"], marker="o", linestyle= "-", color="orange", label="LR")
plt.plot(df_del_nb["perturbation level"], df_del_nb["accuracy"], marker="v", linestyle= "-", color="gray", label="NB")
plt.plot(df_del_rf["perturbation level"], df_del_rf["accuracy"], marker="s", linestyle= "-", color="brown", label="RF")
plt.xlabel("Perturbation level")
plt.ylabel("Accuracy")
plt.title("Deletion")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(0.4, 1)
plt.grid()
plt.show()


# METRICS
# ROBUSTNESS SCORE ALL LR
plt.plot(df_chars_lr["perturbation level"], df_chars_lr["RS"], marker="o", linestyle= "-", color="red", label="Charswap")
plt.plot(df_word_lr["perturbation level"], df_word_lr["RS"], marker="v", linestyle= "-", color="blue", label="WordNet")
plt.plot(df_del_lr["perturbation level"], df_del_lr["RS"], marker="s", linestyle= "-", color="green", label="Deletion")
plt.xlabel("Perturbation level")
plt.ylabel("Robustness Score")
plt.title("RS on Logistic Regression")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(0.4, 1.2)
plt.grid()
plt.show()


# ROBUSTNESS SCORE ALL NB
plt.plot(df_chars_nb["perturbation level"], df_chars_nb["RS"], marker="o", linestyle= "-", color="red", label="Charswap")
plt.plot(df_word_nb["perturbation level"], df_word_nb["RS"], marker="v", linestyle= "-", color="blue", label="WordNet")
plt.plot(df_del_nb["perturbation level"], df_del_nb["RS"], marker="s", linestyle= "-", color="green", label="Deletion")
plt.xlabel("Perturbation level")
plt.ylabel("Robustness Score")
plt.title("RS on Naive Bayes")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(0.4, 1.2)
plt.grid()
plt.show()

# ROBUSTNESS SCORE ALL RF
plt.plot(df_chars_rf["perturbation level"], df_chars_rf["RS"], marker="o", linestyle= "-", color="red", label="Charswap")
plt.plot(df_word_rf["perturbation level"], df_word_rf["RS"], marker="v", linestyle= "-", color="blue", label="WordNet")
plt.plot(df_del_rf["perturbation level"], df_del_rf["RS"], marker="s", linestyle= "-", color="green", label="Deletion")
plt.xlabel("Perturbation level")
plt.ylabel("Robustness Score")
plt.title("RS on Random Forest")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(0.4, 1.2)
plt.grid()
plt.show()

# PDR ALL LR
plt.plot(df_chars_lr["perturbation level"], df_chars_lr["PDR"], marker="o", linestyle= "-", color="red", label="Charswap")
plt.plot(df_word_lr["perturbation level"], df_word_lr["PDR"], marker="v", linestyle= "-", color="blue", label="WordNet")
plt.plot(df_del_lr["perturbation level"], df_del_lr["PDR"], marker="s", linestyle= "-", color="green", label="Deletion")
plt.xlabel("Perturbation level")
plt.ylabel("Performance Drop Rate")
plt.title("PDR on Logistic Regression")
plt.legend(loc="upper left")
plt.xlim(0, 1)
plt.ylim(-0.2, 1)
plt.grid()
plt.show()


# PDR ALL NB
plt.plot(df_chars_nb["perturbation level"], df_chars_nb["PDR"], marker="o", linestyle= "-", color="red", label="Charswap")
plt.plot(df_word_nb["perturbation level"], df_word_nb["PDR"], marker="v", linestyle= "-", color="blue", label="WordNet")
plt.plot(df_del_nb["perturbation level"], df_del_nb["PDR"], marker="s", linestyle= "-", color="green", label="Deletion")
plt.xlabel("Perturbation level")
plt.ylabel("Performance Drop Rate")
plt.title("PDR on Naive Bayes")
plt.legend(loc="upper left")
plt.xlim(0, 1)
plt.ylim(-0.2, 1)
plt.grid()
plt.show()

# PDR ALL RF
plt.plot(df_chars_rf["perturbation level"], df_chars_rf["PDR"], marker="o", linestyle= "-", color="red", label="Charswap")
plt.plot(df_word_rf["perturbation level"], df_word_rf["PDR"], marker="v", linestyle= "-", color="blue", label="WordNet")
plt.plot(df_del_rf["perturbation level"], df_del_rf["PDR"], marker="s", linestyle= "-", color="green", label="Deletion")
plt.xlabel("Perturbation level")
plt.ylabel("Performance Drop Rate")
plt.title("PDR on Random Forest")
plt.legend(loc="upper left")
plt.xlim(0, 1)
plt.ylim(-0.2, 1)
plt.grid()
plt.show()

# MDS ALL LR
plt.plot(df_chars_lr["perturbation level"], df_chars_lr["MDS"], marker="o", linestyle= "-", color="red", label="Charswap")
plt.plot(df_word_lr["perturbation level"], df_word_lr["MDS"], marker="v", linestyle= "-", color="blue", label="WordNet")
plt.plot(df_del_lr["perturbation level"], df_del_lr["MDS"], marker="s", linestyle= "-", color="green", label="Deletion")
plt.xlabel("Perturbation level")
plt.ylabel("Mean Deviaion Score")
plt.title("MDS on Logistic Regression")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(-2, 5)
plt.grid()
plt.show()


# MDS ALL NB
plt.plot(df_chars_nb["perturbation level"], df_chars_nb["MDS"], marker="o", linestyle= "-", color="red", label="Charswap")
plt.plot(df_word_nb["perturbation level"], df_word_nb["MDS"], marker="v", linestyle= "-", color="blue", label="WordNet")
plt.plot(df_del_nb["perturbation level"], df_del_nb["MDS"], marker="s", linestyle= "-", color="green", label="Deletion")
plt.xlabel("Perturbation level")
plt.ylabel("Mean Deviation Score")
plt.title("MDS on Naive Bayes")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(-2, 5)
plt.grid()
plt.show()

# MDS ALL RF
plt.plot(df_chars_rf["perturbation level"], df_chars_rf["MDS"], marker="o", linestyle= "-", color="red", label="Charswap")
plt.plot(df_word_rf["perturbation level"], df_word_rf["MDS"], marker="v", linestyle= "-", color="blue", label="WordNet")
plt.plot(df_del_rf["perturbation level"], df_del_rf["MDS"], marker="s", linestyle= "-", color="green", label="Deletion")
plt.xlabel("Perturbation level")
plt.ylabel("Mean Deviation Score")
plt.title("MDS on Random Forest")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(-2, 5)
plt.grid()
plt.show()


# METRICS ON EACH PERTURBATION TYPE COMPARED TO ACCURACY FOR EACH MODEL
# CHARSWAP LOGISTIC REGRESSION
plt.plot(df_chars_lr["perturbation level"], df_chars_lr["accuracy"], marker=".", linestyle= "--", color="black", label="Accuracy")
plt.plot(df_chars_lr["perturbation level"], df_chars_lr["RS"], marker="o", linestyle= "-", color="purple", label="RS")
plt.plot(df_chars_lr["perturbation level"], df_chars_lr["PDR"], marker="v", linestyle= "-", color="olive", label="PDS")
plt.plot(df_chars_lr["perturbation level"], df_chars_lr["MDS"], marker="s", linestyle= "-", color="red", label="MDS")

plt.xlabel("Perturbation level")
plt.ylabel("Metric value")
plt.title("Charswap LR Metric Comparison")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(-2, 5)
plt.grid()
plt.show()

# CHARSWAP NAIVE BAYES
plt.plot(df_chars_nb["perturbation level"], df_chars_nb["accuracy"], marker=".", linestyle= "--", color="black", label="Accuracy")
plt.plot(df_chars_nb["perturbation level"], df_chars_nb["RS"], marker="o", linestyle= "-", color="purple", label="RS")
plt.plot(df_chars_nb["perturbation level"], df_chars_nb["PDR"], marker="v", linestyle= "-", color="olive", label="PDS")
plt.plot(df_chars_nb["perturbation level"], df_chars_nb["MDS"], marker="s", linestyle= "-", color="red", label="MDS")

plt.xlabel("Perturbation level")
plt.ylabel("Metric value")
plt.title("Charswap NB Metric Comparison")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(-2, 5)
plt.grid()
plt.show()

# CHARSWAP RANDOM FOREST
plt.plot(df_chars_rf["perturbation level"], df_chars_rf["accuracy"], marker=".", linestyle= "--", color="black", label="Accuracy")
plt.plot(df_chars_rf["perturbation level"], df_chars_rf["RS"], marker="o", linestyle= "-", color="purple", label="RS")
plt.plot(df_chars_rf["perturbation level"], df_chars_rf["PDR"], marker="v", linestyle= "-", color="olive", label="PDS")
plt.plot(df_chars_rf["perturbation level"], df_chars_rf["MDS"], marker="s", linestyle= "-", color="red", label="MDS")

plt.xlabel("Perturbation level")
plt.ylabel("Metric value")
plt.title("Charswap RF Metric Comparison")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(-2, 5)
plt.grid()
plt.show()


# WORDNET LOGISTIC REGRESSION
plt.plot(df_word_lr["perturbation level"], df_word_lr["accuracy"], marker=".", linestyle= "--", color="black", label="Accuracy")
plt.plot(df_word_lr["perturbation level"], df_word_lr["RS"], marker="o", linestyle= "-", color="purple", label="RS")
plt.plot(df_word_lr["perturbation level"], df_word_lr["PDR"], marker="v", linestyle= "-", color="olive", label="PDS")
plt.plot(df_word_lr["perturbation level"], df_word_lr["MDS"], marker="s", linestyle= "-", color="red", label="MDS")

plt.xlabel("Perturbation level")
plt.ylabel("Metric value")
plt.title("WordNet LR Metric Comparison")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(-2, 5)
plt.grid()
plt.show()

# WORDNET NAIVE BAYES
plt.plot(df_word_nb["perturbation level"], df_word_nb["accuracy"], marker=".", linestyle= "--", color="black", label="Accuracy")
plt.plot(df_word_nb["perturbation level"], df_word_nb["RS"], marker="o", linestyle= "-", color="purple", label="RS")
plt.plot(df_word_nb["perturbation level"], df_word_nb["PDR"], marker="v", linestyle= "-", color="olive", label="PDS")
plt.plot(df_word_nb["perturbation level"], df_word_nb["MDS"], marker="s", linestyle= "-", color="red", label="MDS")

plt.xlabel("Perturbation level")
plt.ylabel("Metric value")
plt.title("WordNet NB Metric Comparison")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(-2, 5)
plt.grid()
plt.show()

# WORDNET RANDOM FOREST
plt.plot(df_word_rf["perturbation level"], df_word_rf["accuracy"], marker=".", linestyle= "--", color="black", label="Accuracy")
plt.plot(df_word_rf["perturbation level"], df_word_rf["RS"], marker="o", linestyle= "-", color="purple", label="RS")
plt.plot(df_word_rf["perturbation level"], df_word_rf["PDR"], marker="v", linestyle= "-", color="olive", label="PDS")
plt.plot(df_word_rf["perturbation level"], df_word_rf["MDS"], marker="s", linestyle= "-", color="red", label="MDS")

plt.xlabel("Perturbation level")
plt.ylabel("Metric value")
plt.title("WordNet RF Metric Comparison")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(-2, 5)
plt.grid()
plt.show()

# DELETION LOGISTIC REGRESSION
plt.plot(df_del_lr["perturbation level"], df_del_lr["accuracy"], marker=".", linestyle= "--", color="black", label="Accuracy")
plt.plot(df_del_lr["perturbation level"], df_del_lr["RS"], marker="o", linestyle= "-", color="purple", label="RS")
plt.plot(df_del_lr["perturbation level"], df_del_lr["PDR"], marker="v", linestyle= "-", color="olive", label="PDS")
plt.plot(df_del_lr["perturbation level"], df_del_lr["MDS"], marker="s", linestyle= "-", color="red" \
"", label="MDS")

plt.xlabel("Perturbation level")
plt.ylabel("Metric value")
plt.title("Deletion LR Metric Comparison")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(-2, 5)
plt.grid()
plt.show()

# DELETION NAIVE BAYES
plt.plot(df_del_nb["perturbation level"], df_del_nb["accuracy"], marker=".", linestyle= "--", color="black", label="Accuracy")
plt.plot(df_del_nb["perturbation level"], df_del_nb["RS"], marker="o", linestyle= "-", color="purple", label="RS")
plt.plot(df_del_nb["perturbation level"], df_del_nb["PDR"], marker="v", linestyle= "-", color="olive", label="PDS")
plt.plot(df_del_nb["perturbation level"], df_del_nb["MDS"], marker="s", linestyle= "-", color="red", label="MDS")

plt.xlabel("Perturbation level")
plt.ylabel("Metric value")
plt.title("Deletion NB Metric Comparison")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(-2, 5)
plt.grid()
plt.show()

# DELETION RANDOM FOREST
plt.plot(df_del_rf["perturbation level"], df_del_rf["accuracy"], marker=".", linestyle= "--", color="black", label="Accuracy")
plt.plot(df_del_rf["perturbation level"], df_del_rf["RS"], marker="o", linestyle= "-", color="purple", label="RS")
plt.plot(df_del_rf["perturbation level"], df_del_rf["PDR"], marker="v", linestyle= "-", color="olive", label="PDS")
plt.plot(df_del_rf["perturbation level"], df_del_rf["MDS"], marker="s", linestyle= "-", color="red", label="MDS")

plt.xlabel("Perturbation level")
plt.ylabel("Metric value")
plt.title("Deletion RF Metric Comparison")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(-2, 5)
plt.grid()
plt.show()



# TESTING SimpleMDS DELETION RANDOM FOREST
plt.plot(df_del_rf["perturbation level"], df_del_rf["accuracy"], marker=".", linestyle= "--", color="black", label="Accuracy")
plt.plot(df_del_rf["perturbation level"], df_del_rf["MDS"], marker="s", linestyle= "-", color="red", label="MDS")
plt.plot(df_del_rf["perturbation level"], np.array([None, None, 1.857, 0.075, 0.093, 0.851, 0.115, 0.175, 0.079, 0.024, 0.183]), marker="v", linestyle= "-", color="green", label="simpleMDS : acc(n)-acc(n-1)/acc(n-1)-acc(baseline)")
plt.xlabel("Perturbation level")
plt.ylabel("Metric value")
plt.title("MDS and SimpleMDS")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(-1, 2)
plt.grid()
plt.show()


# Comparing MDS, SimpleMDS and MaxMDS on WordNet Naive Bayes
plt.plot(df_word_nb["perturbation level"], df_word_nb["accuracy"], marker=".", linestyle= "--", color="black", label="Accuracy")
plt.plot(df_word_nb["perturbation level"], df_word_nb["MDS"], marker="s", linestyle= "-", color="red", label="MDS")
plt.plot(df_word_nb["perturbation level"], df_word_nb["SimpleMDS"], marker="v", linestyle= "-", color="green", label="SimpleMDS")
plt.plot(df_word_nb["perturbation level"], df_word_nb["MaxMDS"], marker="o", linestyle= "-", color="blue", label="MaxMDS")
plt.xlabel("Perturbation level")
plt.ylabel("Metric value")
plt.title("MDS, SimpleMDS, MaxMDS on NB WordNet accuracy")
plt.legend(loc="lower left")
plt.xlim(0, 1)
plt.ylim(-1, 2)
plt.grid()
plt.show()